In [1]:
!gdown https://drive.google.com/uc?id=18lYAxkxpVyijrzOMvP-x6atSvjKealJ1
!unzip DataSet.zip

Downloading...
From: https://drive.google.com/uc?id=18lYAxkxpVyijrzOMvP-x6atSvjKealJ1
To: /content/DataSet.zip
100% 25.2M/25.2M [00:00<00:00, 56.3MB/s]
Archive:  DataSet.zip
  inflating: dataset.csv             
  inflating: Book1.csv               


In [2]:
#importing Essential libraries
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
import operator
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# from scipy.sparse import csr_matrix
# from pandas.api.types import is_numeric_dtype
# from sklearn.neighbors import NearestNeighbors
# from sklearn.feature_extraction import DictVectorizer
# from sklearn.metrics.pairwise import cosine_similarity
# from sklearn.feature_extraction.text import TfidfVectorizer

import warnings
warnings.filterwarnings("ignore")

In [3]:
#Reading books dataset
books1 = pd.read_csv('dataset.csv')

In [4]:
#Dropping null values
books1 = books1.dropna(axis = 0, subset = ['bookTitle'])

In [5]:
#Dropping duplicates on the basis of book title
books1.drop_duplicates(subset=['bookTitle'], inplace=True)

In [6]:
books1.head()


,url,bookTitle,bookImage,bookAuthors,bookDesc,bookRating,ratingCount,reviewCount,bookPages,bookGenres,bookISBN,recommendations
0,https://www.goodreads.com/book/show/2767052-th...,The Hunger Games,https://i.gr-assets.com/images/S/compressed.ph...,Suzanne Collins,"Could you survive on your own in the wild, wit...",4.32,6717635,176054,374 pages,Young Adult _ Fiction _ Science Fiction-Dystop...,9.780440e+12,"['Divergent (Divergent, #1):https://www.goodre..."
1,https://www.goodreads.com/book/show/2.Harry_Po...,Harry Potter and the Order of the Phoenix,https://i.gr-assets.com/images/S/compressed.ph...,"J.K. Rowling,Mary GrandPrÃ©",There is a door at the end of a silent corrido...,4.50,2668409,45724,870 pages,Fantasy _ Young Adult _ Fiction _ Fantasy-Magi...,NaN,['Harry Potter and the Cursed Child: Parts One...
2,https://www.goodreads.com/book/show/2657.To_Ki...,To Kill a Mockingbird,https://i.gr-assets.com/images/S/compressed.ph...,Harper Lee,The unforgettable novel of a childhood in a sl...,4.28,4772918,95595,324 pages,Classics _ Fiction _ Historical-Historical Fic...,NaN,['The Great Gatsby:https://www.goodreads.com/b...
3,https://www.goodreads.com/book/show/1885.Pride...,Pride and Prejudice,https://i.gr-assets.com/images/S/compressed.ph...,"Jane Austen,Anna Quindlen",Alternate cover edition of ISBN 9780679783268S...,4.27,3206070,74020,279 pages,Classics _ Fiction _ Romance _ Historical-Hist...,NaN,['Jane Eyre:https://www.goodreads.com/book/sho...
4,https://www.goodreads.com/book/show/41865.Twil...,Twilight,https://i.gr-assets.com/images/S/compressed.ph...,Stephenie Meyer,About three things I was absolutely positive.F...,3.61,5231000,107619,501 pages,Young Adult _ Fantasy _ Romance _ Fiction _ Pa...,9.780320e+12,"['The Hunger Games (The Hunger Games, #1):http..."


In [7]:
#Dropping unnecessary columns
books1.drop(['url', 'reviewCount', 'bookPages','bookISBN', 'bookRating', 'ratingCount','recommendations'], axis=1, inplace=True)


In [8]:
books1.head()

,bookTitle,bookImage,bookAuthors,bookDesc,bookGenres
0,The Hunger Games,https://i.gr-assets.com/images/S/compressed.ph...,Suzanne Collins,"Could you survive on your own in the wild, wit...",Young Adult _ Fiction _ Science Fiction-Dystop...
1,Harry Potter and the Order of the Phoenix,https://i.gr-assets.com/images/S/compressed.ph...,"J.K. Rowling,Mary GrandPrÃ©",There is a door at the end of a silent corrido...,Fantasy _ Young Adult _ Fiction _ Fantasy-Magi...
2,To Kill a Mockingbird,https://i.gr-assets.com/images/S/compressed.ph...,Harper Lee,The unforgettable novel of a childhood in a sl...,Classics _ Fiction _ Historical-Historical Fic...
3,Pride and Prejudice,https://i.gr-assets.com/images/S/compressed.ph...,"Jane Austen,Anna Quindlen",Alternate cover edition of ISBN 9780679783268S...,Classics _ Fiction _ Romance _ Historical-Hist...
4,Twilight,https://i.gr-assets.com/images/S/compressed.ph...,Stephenie Meyer,About three things I was absolutely positive.F...,Young Adult _ Fantasy _ Romance _ Fiction _ Pa...


In [9]:
books1.dropna(inplace=True)

In [10]:
books1['bookGenres'] = books1['bookGenres'].str.split('_')

In [11]:
books1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9339 entries, 0 to 9999
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   bookTitle    9339 non-null   object
 1   bookImage    9339 non-null   object
 2   bookAuthors  9339 non-null   object
 3   bookDesc     9339 non-null   object
 4   bookGenres   9339 non-null   object
dtypes: object(5)
memory usage: 437.8+ KB


In [12]:
books1.head()

,bookTitle,bookImage,bookAuthors,bookDesc,bookGenres
0,The Hunger Games,https://i.gr-assets.com/images/S/compressed.ph...,Suzanne Collins,"Could you survive on your own in the wild, wit...","[Young Adult , Fiction , Science Fiction-Dys..."
1,Harry Potter and the Order of the Phoenix,https://i.gr-assets.com/images/S/compressed.ph...,"J.K. Rowling,Mary GrandPrÃ©",There is a door at the end of a silent corrido...,"[Fantasy , Young Adult , Fiction , Fantasy-..."
2,To Kill a Mockingbird,https://i.gr-assets.com/images/S/compressed.ph...,Harper Lee,The unforgettable novel of a childhood in a sl...,"[Classics , Fiction , Historical-Historical ..."
3,Pride and Prejudice,https://i.gr-assets.com/images/S/compressed.ph...,"Jane Austen,Anna Quindlen",Alternate cover edition of ISBN 9780679783268S...,"[Classics , Fiction , Romance , Historical-..."
4,Twilight,https://i.gr-assets.com/images/S/compressed.ph...,Stephenie Meyer,About three things I was absolutely positive.F...,"[Young Adult , Fantasy , Romance , Fiction ..."


In [13]:
books1['bookDesc'] = books1['bookDesc'].apply(lambda x:x.split())
books1['bookAuthors'] = books1['bookAuthors'].apply(lambda x:x.split())

In [14]:
books1.head()

,bookTitle,bookImage,bookAuthors,bookDesc,bookGenres
0,The Hunger Games,https://i.gr-assets.com/images/S/compressed.ph...,"[Suzanne, Collins]","[Could, you, survive, on, your, own, in, the, ...","[Young Adult , Fiction , Science Fiction-Dys..."
1,Harry Potter and the Order of the Phoenix,https://i.gr-assets.com/images/S/compressed.ph...,"[J.K., Rowling,Mary, GrandPrÃ©]","[There, is, a, door, at, the, end, of, a, sile...","[Fantasy , Young Adult , Fiction , Fantasy-..."
2,To Kill a Mockingbird,https://i.gr-assets.com/images/S/compressed.ph...,"[Harper, Lee]","[The, unforgettable, novel, of, a, childhood, ...","[Classics , Fiction , Historical-Historical ..."
3,Pride and Prejudice,https://i.gr-assets.com/images/S/compressed.ph...,"[Jane, Austen,Anna, Quindlen]","[Alternate, cover, edition, of, ISBN, 97806797...","[Classics , Fiction , Romance , Historical-..."
4,Twilight,https://i.gr-assets.com/images/S/compressed.ph...,"[Stephenie, Meyer]","[About, three, things, I, was, absolutely, pos...","[Young Adult , Fantasy , Romance , Fiction ..."


In [15]:
books1.isnull().sum() 

bookTitle      0
bookImage      0
bookAuthors    0
bookDesc       0
bookGenres     0
dtype: int64

In [16]:
books1['comb'] = books1['bookAuthors'] + books1['bookGenres']

In [17]:
books1.head()

,bookTitle,bookImage,bookAuthors,bookDesc,bookGenres,comb
0,The Hunger Games,https://i.gr-assets.com/images/S/compressed.ph...,"[Suzanne, Collins]","[Could, you, survive, on, your, own, in, the, ...","[Young Adult , Fiction , Science Fiction-Dys...","[Suzanne, Collins, Young Adult , Fiction , S..."
1,Harry Potter and the Order of the Phoenix,https://i.gr-assets.com/images/S/compressed.ph...,"[J.K., Rowling,Mary, GrandPrÃ©]","[There, is, a, door, at, the, end, of, a, sile...","[Fantasy , Young Adult , Fiction , Fantasy-...","[J.K., Rowling,Mary, GrandPrÃ©, Fantasy , You..."
2,To Kill a Mockingbird,https://i.gr-assets.com/images/S/compressed.ph...,"[Harper, Lee]","[The, unforgettable, novel, of, a, childhood, ...","[Classics , Fiction , Historical-Historical ...","[Harper, Lee, Classics , Fiction , Historica..."
3,Pride and Prejudice,https://i.gr-assets.com/images/S/compressed.ph...,"[Jane, Austen,Anna, Quindlen]","[Alternate, cover, edition, of, ISBN, 97806797...","[Classics , Fiction , Romance , Historical-...","[Jane, Austen,Anna, Quindlen, Classics , Fict..."
4,Twilight,https://i.gr-assets.com/images/S/compressed.ph...,"[Stephenie, Meyer]","[About, three, things, I, was, absolutely, pos...","[Young Adult , Fantasy , Romance , Fiction ...","[Stephenie, Meyer, Young Adult , Fantasy , R..."


In [18]:
books1['bookTitle'] = books1['bookTitle'].str.lower()

In [19]:
books1.to_csv('books1.csv', index = False)

In [20]:
books1.shape

(9339, 6)

In [21]:
books1.isna().sum()

bookTitle      0
bookImage      0
bookAuthors    0
bookDesc       0
bookGenres     0
comb           0
dtype: int64

In [22]:
def remove_spaces(row):
    x=[]
    for i in row:
        i=i.replace(' ','')
        i=i.lower()
        x.append(i)
    return(x)

In [23]:
books1['bookGenres'] = books1['bookGenres'].apply(remove_spaces).str.join(' ')

In [24]:
books1['bookDesc'] = books1['bookDesc'].apply(remove_spaces).str.join(' ')

In [25]:
books1['bookAuthors'] = books1['bookAuthors'].apply(remove_spaces).str.join(' ')

In [26]:
books1['comb'] = books1['comb'].apply(remove_spaces).str.join(' ')

In [27]:
books1.head()

,bookTitle,bookImage,bookAuthors,bookDesc,bookGenres,comb
0,the hunger games,https://i.gr-assets.com/images/S/compressed.ph...,suzanne collins,"could you survive on your own in the wild, wit...",youngadult fiction sciencefiction-dystopia fan...,suzanne collins youngadult fiction sciencefict...
1,harry potter and the order of the phoenix,https://i.gr-assets.com/images/S/compressed.ph...,"j.k. rowling,mary grandprã©",there is a door at the end of a silent corrido...,fantasy youngadult fiction fantasy-magic child...,"j.k. rowling,mary grandprã© fantasy youngadult..."
2,to kill a mockingbird,https://i.gr-assets.com/images/S/compressed.ph...,harper lee,the unforgettable novel of a childhood in a sl...,classics fiction historical-historicalfiction ...,harper lee classics fiction historical-histori...
3,pride and prejudice,https://i.gr-assets.com/images/S/compressed.ph...,"jane austen,anna quindlen",alternate cover edition of isbn 9780679783268s...,classics fiction romance historical-historical...,"jane austen,anna quindlen classics fiction rom..."
4,twilight,https://i.gr-assets.com/images/S/compressed.ph...,stephenie meyer,about three things i was absolutely positive.f...,youngadult fantasy romance fiction paranormal-...,stephenie meyer youngadult fantasy romance fic...


books1.head()

In [28]:
!pip install rake-nltk
import nltk
nltk.download('punkt')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.5 MB 18.3 MB/s 
     |████████████████████████████████| 749 kB 49.6 MB/s 
  Attempting uninstall: regex
    Found existing installation: regex 2019.12.20
    Uninstalling regex-2019.12.20:
      Successfully uninstalled regex-2019.12.20
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [29]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [30]:
from rake_nltk import Rake
r = Rake()
def extract_important_words(row):
    word = r.extract_keywords_from_text(row)
    word = r.get_ranked_phrases()
    return ' '.join(word)

In [31]:
#Removing english stopwords
books1['bookDesc'] = books1['bookDesc'].apply(extract_important_words)

In [32]:
books1['bookDesc']

0       without really meaning start making choices no...
1       personality like poisoned honey ordinary wizar...
2       young alabama woman claims universal appeal ha...
3       radiant wit sparkles alternate cover edition j...
4       part might beâ €” himâ €” three things extraor...
                              ...                        
9995    civil war 1 whose side single misstep marvel u...
9996    volatile feudal tsar stupendous russian navy m...
9997    cozy little house snowy night new friend invit...
9998    young doctor called norton perina signs things...
9999    blockbuster novels five point someone bihari b...
Name: bookDesc, Length: 9339, dtype: object

In [33]:
books1['comb'] = books1['bookAuthors'] + books1['bookGenres'] + books1['bookDesc']

In [34]:
def clear(data):
    result=[]
    result.append(re.sub("[^a-z1-9]","",data))
    return result

In [35]:
books1['title'] = books1['bookTitle'].apply(clear).str.join('')

In [36]:
books1.head()

,bookTitle,bookImage,bookAuthors,bookDesc,bookGenres,comb,title
0,the hunger games,https://i.gr-assets.com/images/S/compressed.ph...,suzanne collins,without really meaning start making choices no...,youngadult fiction sciencefiction-dystopia fan...,suzanne collinsyoungadult fiction scienceficti...,thehungergames
1,harry potter and the order of the phoenix,https://i.gr-assets.com/images/S/compressed.ph...,"j.k. rowling,mary grandprã©",personality like poisoned honey ordinary wizar...,fantasy youngadult fiction fantasy-magic child...,"j.k. rowling,mary grandprã©fantasy youngadult ...",harrypotterandtheorderofthephoenix
2,to kill a mockingbird,https://i.gr-assets.com/images/S/compressed.ph...,harper lee,young alabama woman claims universal appeal ha...,classics fiction historical-historicalfiction ...,harper leeclassics fiction historical-historic...,tokillamockingbird
3,pride and prejudice,https://i.gr-assets.com/images/S/compressed.ph...,"jane austen,anna quindlen",radiant wit sparkles alternate cover edition j...,classics fiction romance historical-historical...,"jane austen,anna quindlenclassics fiction roma...",prideandprejudice
4,twilight,https://i.gr-assets.com/images/S/compressed.ph...,stephenie meyer,part might beâ €” himâ €” three things extraor...,youngadult fantasy romance fiction paranormal-...,stephenie meyeryoungadult fantasy romance fict...,twilight


In [37]:
books1.to_csv('new_data.csv',index=False)

In [ ]:
from google.colab import files
files.download('new_data.csv')